In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from py2neo import Graph, Node, Relationship
import nltk
from py2neo import *
import pandas as pd
from pprint import pprint
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import spacy
from spacy import displacy
import en_core_web_sm
from nltk import pos_tag
from itertools import *
import glob
import csv
from gensim.matutils import softcossim
from gensim import corpora
import gensim
import time
import gensim.downloader as api
from gensim.matutils import softcossim
from gensim import corpora
import gensim
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("C:/Users/wongzn/Desktop/rs_data_coref_nomax.csv")[["relation","subject","object"]]

In [3]:
df.head()

,relation,subject,object
0,was,21-year-old Malaysian suspect,arrested
1,containing,he,336g of Ice
2,containing,he,336g
3,containing 336g at,he,carpark in vicinity of Orchard Towers
4,was,Malaysian suspect,arrested


# Cleaning Relations Data

In [5]:
df.drop_duplicates(inplace=True) #dropping duplicated relations
df.reset_index(drop=True,inplace=True)
df.all = df.relation + ' ' + df.subject +' '+ df.object

In [6]:
fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')

In [7]:
documents = df.all.apply(lambda x: x.split())
dictionary = corpora.Dictionary(documents)

In [8]:
st=time.time()
similarity_matrix = fasttext_model300.similarity_matrix(dictionary, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)
time.time()-st

2698.08948802948

In [9]:
sentences = documents.apply(lambda x: dictionary.doc2bow(x))
sent = pd.DataFrame(sentences)

In [ ]:
#Loop through Relations data, remove Relations that are similar

index = []
st = time.time()

for i in tqdm(range(len(sent))):
    if i not in index:
        for j in range(15):
            scs = softcossim(sent[0][i],sent[0][i+j+1],similarity_matrix)
            if scs > 0.75: #Threshold
                len1 = len(sent[0][i])
                len2 = len(sent[0][i+j+1])
                if len1 > len2: #Getting index of the shorter relation
                    index.append(i+j+1)
                else:
                    index.append(i)
            else:
                continue
    else: 
        continue

In [11]:
print(time.time()-st)

2360.408665895462


In [12]:
index = list(set(index))
df.drop(index,axis=0,inplace=True) #Removing Relations that are similar and shorter in length
df.reset_index(drop=True,inplace=True)

In [13]:
df.drop_duplicates(inplace=True) #Drop duplicates again just in case

In [14]:
df = df[['relation','subject','object']]

In [15]:
df['subject'] = df['subject'].apply(lambda x: re.sub('\n','',x))

In [16]:
#Get unique subjects
subj = df.subject.unique()

In [17]:
len(subj)

29466

In [31]:
#df.to_csv("C:/Users/wongzn/Desktop/df_backupafterprocess.csv")

# Further Cleaning of Relations By Subject

In [50]:
subj_1 = subj[:9879]
subj_2 = subj[9879:14879]
subj_3 = subj[14879:19879]
subj_4 = subj[19879:24879]
subj_5 = subj[24879:]

In [ ]:
#final_df = pd.DataFrame(columns=['relation','subject','object'])

for s in tqdm(subj):
    new_df = df[df.subject == s]
    new_df.reset_index(inplace=True,drop=True)
    new_df.all = new_df.relation + ' ' + new_df.subject +' '+ new_df.object
    documents = new_df.all.apply(lambda x: x.split())
    dictionary = corpora.Dictionary(documents)
    similarity_matrix = fasttext_model300.similarity_matrix(dictionary, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)
    sentences = documents.apply(lambda x: dictionary.doc2bow(x))
    sent = pd.DataFrame(sentences)
    index = []
    for i in range(len(sent)):
        if i not in index:
            for j in range(5):
                try:
                    scs = softcossim(sent[0][i],sent[0][i+j+1],similarity_matrix)
                    if scs > 0.75:
                        len1 = len(sent[0][i])
                        len2 = len(sent[0][i+j+1])
                        if len1 > len2:
                            index.append(i+j+1)
                        else:
                            index.append(i)
                            
                except:
                    continue
    index = list(set(index))
    new_df.drop(index,axis=0,inplace=True)
    final_df = final_df.append(new_df, ignore_index=True)

In [20]:
final_df

,relation,subject,object
0,was,malaysian suspect,arrested
1,is in,carpark,vicinity of orchard towers
2,is in,carpark,sengkang housing estate
3,is in,carpark,bukit ho swee
4,is in,carpark,kio
5,is in,carpark,toa payoh lorong
6,is in,carpark,clementi avenue
7,be,carpark,will closed
8,is in,carpark,commonwealth drive
9,from officers is,central narcotics bureau,cnb


In [26]:
#FINAL = pd.DataFrame(columns=['relation','subject','object'])
#FINAL = FINAL.append(final_df, ignore_index=True)
#final_df.to_csv("C:/Users/wongzn/Desktop/Final_Cleaned_Relations.csv")

In [21]:
import numpy as np
final_df['subject'].replace('', np.nan, inplace=True)
final_df['subject'].replace(' ', np.nan, inplace=True)
final_df['subject'].replace('  ', np.nan, inplace=True)
final_df['subject'].replace('   ', np.nan, inplace=True)
final_df['subject'].replace('    ', np.nan, inplace=True)
final_df['subject'].replace('     ', np.nan, inplace=True)
final_df['subject'].replace('      ', np.nan, inplace=True)
final_df['subject'].replace('       ', np.nan, inplace=True)
final_df['subject'].replace('        ', np.nan, inplace=True)
final_df['subject'].replace('         ', np.nan, inplace=True)
final_df['subject'].replace('          ', np.nan, inplace=True)
final_df['subject'].replace('           ', np.nan, inplace=True)
final_df['subject'].replace('            ', np.nan, inplace=True)

In [22]:
final_df['subject'].dropna(inplace=True)
final_df['subject'] = final_df['subject'].apply(lambda x: x.strip())

In [25]:
sum(final_df.subject.value_counts()>3)

3694

In [24]:
final_df.drop_duplicates(inplace=True)

# Importing into Neo4j

In [27]:
import nltk
from py2neo import *
import pandas as pd
from pprint import pprint
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import spacy
from spacy import displacy
import en_core_web_sm
from nltk import pos_tag
from itertools import *
import glob
import csv
from py2neo import Graph, Node, Relationship

In [28]:
graph = Graph('bolt://localhost:7687', auth=("neo4j","123"))
tx = graph.begin()

In [29]:
subject_df = list(final_df.subject.unique())

In [36]:
for i in subject_df:
    new_df = final_df[final_df.subject == i]
    new_df.reset_index(drop=True,inplace=True)
    N1 = Node("Subject", subj=i)
    #tx.merge(N1,"dat","data")
    for row in range(len(new_df)):
        N2 = Node("Object", obj=new_df.object[row])
        #tx.merge(N2,"Object","obj")
        rel = Relationship(N1, new_df.relation[row], N2)
        graph.create(rel)

# Steps to set up Neo4j

1. Download GraphAware files (for NLP): https://products.graphaware.com/
        - Installing GA on Neo4j: https://github.com/graphaware/neo4j-nlp
2. Download StanfordCoreNLP Jar file: https://stanfordnlp.github.io/CoreNLP/download.html
3. Install APOC Plugin on Neo4j
4. Run the above Python code and Neo4j Query to merge the nodes

 # Neo4j Codes

### Running this query will give us the following:

![Querying A Specific Subject](https://user-images.githubusercontent.com/36468263/77022937-d1598480-69c5-11ea-9862-51e881071495.jpg)

In [ ]:
Double click to Expand

![Expanding A Particular Node By Double Clicking](https://user-images.githubusercontent.com/36468263/77022939-d28ab180-69c5-11ea-9e54-6ef75fa6011f.jpg)

In [ ]:
Double click on PM

![Subjects Linked To Prime Ministers](https://user-images.githubusercontent.com/36468263/77022942-d3234800-69c5-11ea-8862-1975e3f02c4e.jpg)

In [ ]:
Double Click on Jewel

![Expanding Jewel Changi](https://user-images.githubusercontent.com/36468263/77022850-9ce5c880-69c5-11ea-8eee-112f6553584f.jpg)

### To Query For Relations